In [1]:
import numpy as np
import torch
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
import argparse
import logging
import os
import heapq
import random
import pickle
import sys
import pandas


In [2]:
param_list = []
for i in range(1,3):
    print(i)
    param_list.append(pickle.load(open("../data/params"+str(i)+".pkl","rb")))

1
2


In [3]:
param_list

[[100, 604, tensor(196.), 36.0, 2.108670177953863, 3.468405773832637],
 [100, 589, tensor(196.), 36.0, 4.945287403126166, 2.7330913264429393]]

In [4]:
W_jets = []
QCD_jets = []
Jets = []
for i,params in enumerate(param_list):
    print(i+1,params)
    jetP = params[1]
    M2start = params[2]
    pt_cut = params[3]
    rate = params[4]
    W_rate = params[5]
    
    QCD_aux = pickle.load(open("../data/QCD_jets"+str(i+1)+"_jetp_"+str(jetP)+"_lambda_"+str(round(rate,3))+"_ptcut_"+str(round(float(pt_cut),3))+".pkl","rb"))
    QCD_jets.append(QCD_aux)
    for j, jet in enumerate(QCD_aux):
        Jets.append(jet)
    W_aux = pickle.load(open("../data/W_jets"+str(i+1)+"_jetp_"+str(jetP)+"_lambda_"+str(round(rate,3))+"_ptcut_"+str(round(float(pt_cut),3))+"W_rate"+str(W_rate)+".pkl","rb"))
    W_jets.append(W_aux)
    for j, jet in enumerate(W_aux):
        Jets.append(jet)


num_jets = len(Jets)
print(num_jets) 

1 [100, 604, tensor(196.), 36.0, 2.108670177953863, 3.468405773832637]
2 [100, 589, tensor(196.), 36.0, 4.945287403126166, 2.7330913264429393]
400


In [25]:
pickle.dump(Jets,open('code/top_reference_dataset/in_data/test.pkl', "wb"),protocol=2)
Jets[1].keys()

dict_keys(['root_id', 'tree', 'content', 'LambdaRoot', 'Lambda', 'Delta_0', 'pt_cut', 'algorithm', 'deltas', 'draws', 'leaves', 'M_Hard', 'logLH', 'dij', 'ConstPhi', 'PhiDelta', 'PhiDeltaRel'])

In [5]:
def plot_hist(bin_num, jets_1,jets_2,titl,jetType):
    N_1 = [len(jet["leaves"]) for jet in jets_1]
    N_2 = [len(jet["leaves"]) for jet in jets_2]
    minBin = np.min(np.concatenate((N_1,N_2)))
    maxBin = np.max(np.concatenate((N_1,N_2)))
    bin_edges = np.arange(minBin-1,maxBin+1,bin_num)
    print(minBin,maxBin,bin_edges)
    fig2, (axes) = plt.subplots(nrows = 1, ncols = 1)
    fig2.set_size_inches(10,5)
    plt.tight_layout(pad= 0.4, w_pad = 5, h_pad = 1.0)
    if jetType == 'Same':
        axes.hist( N_1, histtype = 'step',bins = bin_edges, edgecolor = "c", fill = False, align = 'mid', label = "(lam, ptc) = "+str(get_params(jets_1,"Lambda", "pt_cut")), density = True)
        axes.hist(N_2, histtype = 'step',bins = bin_edges,edgecolor = "m", fill = False,align = 'mid', label = "(lam, ptc) = "+str(get_params(jets_2,"Lambda", "pt_cut")),density = True)
    else:
        axes.hist(N_1, histtype = 'step',bins = bin_edges, edgecolor = "c",fill = False, align = 'mid', label = r"$H_0$", density = True)
        axes.hist(N_2, histtype = 'step',bins = bin_edges,edgecolor = "m",fill = False,align = 'mid', label =r"$H_1$",density = True)

    axes.set_xlabel(r"$N_{leaves}$", fontsize = 15)
    axes.set_ylabel(r"$p\left(N_{leaves}\right)$", fontsize = 15)

    axes.legend(loc = 'best', fontsize = 15)
    axes.grid(False)
 #   titl = '(pt_cut, rr) = '+ str(par)
    axes.set_title(titl)




In [6]:
plot_hist(1,QCD_jets[1],W_jets[1],' ',"Diff")
plot_hist(1,QCD_jets[0],W_jets[0],' ',"Diff")


2 4 [1 2 3 4]
2 6 [1 2 3 4 5 6]


# Reshuffle data and save in train, dev, and test sets
### replaces preprocessing for now (no alignment with phi=0, etc.) 

In [26]:
##Flatten Lists 
num = num_jets//2
W_flat = [item for sublist in W_jets for item in sublist]
QCD_flat = [item for sublist in QCD_jets for item in sublist]
el = np.arange(num)
random.shuffle(el) #Shuffle 
W_shuff = [W_flat[el[i]] for i in range(len(el))]
QCD_shuff = [QCD_flat[el[i]] for i in range(len(el))]

In [27]:
##Sorting into train, val, and test sets

n_train = 3*num//5
n_val = num//5
n_test = n_val
print(n_train+n_val+n_test==num)

train =W_shuff[0:n_train]+QCD_shuff[0:n_train]
val = W_shuff[n_train+1:n_train+1+n_val]+QCD_shuff[n_train+1:n_train+1+n_val]
test = W_shuff[n_train+2+n_val:n_train+2+n_val+n_test]+QCD_shuff[n_train+2+n_val:n_train+2+n_val+n_test]

True


In [ ]:
pickle.dump(train,open('/Users/laurengreenspan/Dropbox/JetTree/TreeNiNLG/code/top_reference_dataset/in_data/train.pkl', "wb"),protocol=2)
pickle.dump(val,open('/Users/laurengreenspan/Dropbox/JetTree/TreeNiNLG/code/top_reference_dataset/in_data/val.pkl', "wb"),protocol=2)
pickle.dump(test,open('/Users/laurengreenspan/Dropbox/JetTree/TreeNiNLG/code/top_reference_dataset/in_data/test.pkl', "wb"),protocol=2)


# Compare with toptag data


#### Comparing ginkgo dictionaries to those of pre-processed jets

In [83]:
ginkgo_test = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/raw_trees/ginkgo_10_jets_test.pkl',"rb"))
Reclustered_test = pickle.load(open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/reclustered_trees/reclustered_10jets_test.pkl',"rb"))

trellis_test = pickle.load(open('/Users/laurengreenspan/Dropbox/JetTree/data/trellis_10_jets_test','rb'))

trellis_test[2]


[{'Z': [-19.51880448893779],
  'trellis_MLE': [-20.466739116998035],
  'RunTime': [0.016156911849975586],
  'Ntrees': [15.0],
  'totTrees': 1,
  'Nleaves': [4],
  'gt_llh': [-25.248253433580604]},
 {'Z': [-20.00970823028727],
  'trellis_MLE': [-21.073448928925934],
  'RunTime': [0.017107248306274414],
  'Ntrees': [15.0],
  'totTrees': 1,
  'Nleaves': [4],
  'gt_llh': [-25.248253433580604]}]

In [58]:
ginkgo_keys = ginkgo_test[0].keys()
TreeNiN_keys = Reclustered_test[0][0].keys()

print(ginkgo_keys)
print(TreeNiN_keys)

print([(Reclustered_test[i][0]['content'],ginkgo_test[i]['content']) for i in range(len(Reclustered_test))][0])

ginkgo_test[0]['leaves'][0]

dict_keys(['root_id', 'tree', 'content', 'LambdaRoot', 'Lambda', 'Delta_0', 'pt_cut', 'algorithm', 'deltas', 'draws', 'leaves', 'M_Hard', 'logLH', 'dij', 'ConstPhi', 'PhiDelta', 'PhiDeltaRel'])
dict_keys(['root_id', 'tree', 'content', 'mass', 'pt', 'energy', 'eta', 'phi'])
(array([[230.94012312, 230.94012336, 230.94012409, 401.12344064],
       [225.41658565, 226.32629416, 222.81322118, 390.25866201],
       [179.00344387, 174.19877373, 169.76143509, 302.35334089],
       [124.01684594, 120.25125398, 111.51363613, 205.76890678],
       [ 74.04012761,  74.71065981,  69.14954786, 125.95558315],
       [ 60.96763641,  60.75446465,  56.31661475, 102.91303267],
       [ 13.0724912 ,  13.95619516,  12.83293311,  23.04255048],
       [ 49.97671832,  45.54059417,  42.36408826,  79.81332363],
       [ 54.98659793,  53.94751975,  58.24779896,  96.58443411],
       [ 46.41314178,  52.12752042,  53.05178609,  87.90532112],
       [ 35.8604571 ,  39.96187871,  40.59728283,  67.51756733],
       [ 1

array([67.51756733, 35.8604571 , 39.96187871, 40.59728283])

In [75]:
dict_keys(['root_id', 'tree', 'content', 'mass', 'pt', 'energy', 'eta', 'phi'])


new_dict = {}
Pre_Jets1 = []
jet = ginkgo_test[1]

label = 0
p = ((jet['content'][0,0:3]**2).sum())**0.5

px = jet['content'][0,1]
py = jet['content'][0,2]
px = jet['content'][0,3]

        
pT = np.asarray([np.linalg.norm(const[1:3]) for const in jet["leaves"]])
        
eta = 0.5 * (np.log(p+pz) - np.log(p-pz))
phi = np.arctan2(py,px)

new_dict['phi']=phi
new_dict['pt']=pT
new_dict['root_id']=jet['root_id']
new_dict['energy']=jet['content'][0,0]
new_dict['tree']=jet['tree']
new_dict['content']=jet['content']##Ordering depends on pytorch or tf implementation
new_dict['eta']=eta
new_dict['mass']=jet['M_Hard']

new_dict['truthLH']=jet['logLH']

entry_test = 'energy'
print(new_dict[entry_test],Reclustered_test[1][0][entry_test])
output = (new_dict,label)

401.1234130859375 401.1234451388827


In [5]:
import sys
#sys.path.append("/Users/laurengreenspan/fastjet-install/lib/python3.8/site-packages")
import fastjet as fj
fj.PseudoJet(1,1,1,1)

<fastjet.PseudoJet; proxy of <Swig Object of type 'fastjet::PseudoJet *' at 0x7ff55eb635d0> >